# 缺失数据

In [1]:
import numpy as np
import pandas as pd

## 缺失值的统计和删除

### 缺失信息的统计

In [2]:
df = pd.read_csv('data/learn_pandas.csv',
                 usecols = ['Grade', 'Name', 'Gender', 'Height',
                            'Weight', 'Transfer'])

可以使用`isna`或`isnull`函数来查看**每个**单元格是否缺失：

In [3]:
df.isna().head()

,Grade,Name,Gender,Height,Weight,Transfer
0,False,False,False,False,False,False
1,False,False,False,False,False,False
2,False,False,False,False,False,False
3,False,False,False,True,False,False
4,False,False,False,False,False,False


In [4]:
df.isnull().head()

,Grade,Name,Gender,Height,Weight,Transfer
0,False,False,False,False,False,False
1,False,False,False,False,False,False
2,False,False,False,False,False,False
3,False,False,False,True,False,False
4,False,False,False,False,False,False


In [5]:
# 结合mean函数可以计算每列缺失的比例
df.isna().mean()

Grade       0.000
Name        0.000
Gender      0.000
Height      0.085
Weight      0.055
Transfer    0.060
dtype: float64

可以利用Series上的`isna`或`notna`来生成布尔索引，查看某一列缺失或非缺失的行：

In [6]:
# 查看身高缺失的行
df[df.Height.isna()].head()

,Grade,Name,Gender,Height,Weight,Transfer
3,Sophomore,Xiaojuan Sun,Female,NaN,41.0,N
12,Senior,Peng You,Female,NaN,48.0,NaN
26,Junior,Yanli You,Female,NaN,48.0,N
36,Freshman,Xiaojuan Qin,Male,NaN,79.0,Y
60,Freshman,Yanpeng Lv,Male,NaN,65.0,N


若要考察多列是否全部缺失（非缺失）或至少有一个缺失（非缺失），则可以在`isna`和`notna`的基础上结合`any`和`all`：

In [7]:
# 找出身高、体重和是否转系均为缺失的个体
sub_set = df[['Height','Weight','Transfer']]

df[sub_set.isna().all(1)]

,Grade,Name,Gender,Height,Weight,Transfer
102,Junior,Chengli Zhao,Male,NaN,NaN,NaN


### 缺失信息的删除

可以利用`dropna`函数来完成缺失值删除的操作，该函数的关键参数有：`axis`（轴方向，默认为0，即删除行）、`how`（删除方式，包括`any`和`all`）、`thresh`（通常在删除列时设置，非缺失值若没有达到这个数量即被删除）和`subset`（备选的删除子集）。

In [8]:
# 删除身高或体重缺失的行
res = df.dropna(how='any', subset=['Height','Weight'])
res.shape

(174, 6)

In [9]:
# 删除缺失值超过15个的列
res = df.dropna(1, thresh=df.shape[0]-15)
res.head()
# 可以看到，身高一列被删除了

,Grade,Name,Gender,Weight,Transfer
0,Freshman,Gaopeng Yang,Female,46.0,N
1,Freshman,Changqiang You,Male,70.0,N
2,Senior,Mei Sun,Male,89.0,N
3,Sophomore,Xiaojuan Sun,Female,41.0,N
4,Sophomore,Gaojuan You,Male,74.0,N


## 缺失值的填充和插值

### 利用fillna进行填充

`fillna`函数有三个常用参数：`value`（填充值，可以是标量，也可以构建索引到元素字典映射）、`method`（填充方法，包括向前填充`ffill`和向后填充`bfill`）和`limit`（连续缺失值最大填充次数）。

In [10]:
# 生成示例
s = pd.Series([np.nan, 1, np.nan, np.nan, 2, np.nan],list('aaabcd'))

In [11]:
s

a    NaN
a    1.0
a    NaN
b    NaN
c    2.0
d    NaN
dtype: float64

In [12]:
# 向前填充
s.fillna(method='ffill')

a    NaN
a    1.0
a    1.0
b    1.0
c    2.0
d    2.0
dtype: float64

In [13]:
# 向后填充
s.fillna(method='bfill')

a    1.0
a    1.0
a    2.0
b    2.0
c    2.0
d    NaN
dtype: float64

In [14]:
# value为标量
s.fillna(s.mean())

a    1.5
a    1.0
a    1.5
b    1.5
c    2.0
d    1.5
dtype: float64

In [15]:
# 连续出现的缺失，最多填充一次
s.fillna(method='ffill',limit=1)

a    NaN
a    1.0
a    1.0
b    NaN
c    2.0
d    2.0
dtype: float64

In [16]:
# 通过索引映射填充的值
s.fillna({'a':100,'d':200})

a    100.0
a      1.0
a    100.0
b      NaN
c      2.0
d    200.0
dtype: float64

需要注意的是，有时为了更合理地填充，需要先进行分组然后再操作。

In [17]:
# 根据年级进行身高的均值填充
df.groupby('Grade')['Height'].transform(lambda x: x.fillna(x.mean())).head()

0    158.900000
1    166.500000
2    188.900000
3    163.075862
4    174.000000
Name: Height, dtype: float64

#### 练一练1

In [18]:
s = pd.Series([1,np.nan,3,5,np.nan,np.nan,4,np.nan,6])
s

0    1.0
1    NaN
2    3.0
3    5.0
4    NaN
5    NaN
6    4.0
7    NaN
8    6.0
dtype: float64

我的实现方法如下，似乎没有用到`limit`参数：

In [19]:
s.fillna((s.shift(1)+s.shift(-1))/2)

0    1.0
1    2.0
2    3.0
3    5.0
4    NaN
5    NaN
6    4.0
7    5.0
8    6.0
dtype: float64

### 插值函数

`interpolate`插值函数的常用参数包括：`method`（插值方法）、`limit_direction`（控制方向，`forward` / `backward` / `both`）、`limit`（控制最大连续缺失值个数）。

常用的简单插值方法包括：线性插值`linear`（默认）、最近邻插值`nearest`和索引插值`index`。

In [20]:
s = pd.Series([np.nan, np.nan, 1,np.nan, np.nan, np.nan,2, np.nan, np.nan])

In [21]:
# 默认为线性插值差值
s.interpolate()

0     NaN
1     NaN
2    1.00
3    1.25
4    1.50
5    1.75
6    2.00
7    2.00
8    2.00
dtype: float64

In [22]:
# 改为向后差值
s.interpolate(limit_direction='backward')

0    1.00
1    1.00
2    1.00
3    1.25
4    1.50
5    1.75
6    2.00
7     NaN
8     NaN
dtype: float64

In [23]:
# 改为双向差值
s.interpolate(limit_direction='both')

0    1.00
1    1.00
2    1.00
3    1.25
4    1.50
5    1.75
6    2.00
7    2.00
8    2.00
dtype: float64

In [24]:
s.interpolate('nearest')

0    NaN
1    NaN
2    1.0
3    1.0
4    1.0
5    2.0
6    2.0
7    NaN
8    NaN
dtype: float64

索引差值会根据**索引大小**进行线性插值：

In [25]:
s = pd.Series([0,np.nan,10],index=[0,1,10])
s

0      0.0
1      NaN
10    10.0
dtype: float64

In [26]:
# 默认
s.interpolate()

0      0.0
1      5.0
10    10.0
dtype: float64

In [27]:
# 索引差值
s.interpolate('index')

0      0.0
1      1.0
10    10.0
dtype: float64

## Nullable类型

### 缺失记号及其缺陷

`numpy`中的缺失值为`np.nan`，它是浮点型数据：

In [28]:
type(np.nan)

float

以浮点型数据存放缺失值很可能生成`object`混合类型对象，这样的效果是不理想的：

In [29]:
# 整数序列加入浮点型缺失值后，改为浮点型序列
pd.Series([1,np.nan]).dtype

dtype('float64')

In [30]:
# 布尔序列加入浮点型缺失值后，改为Object
pd.Series([True,False,np.nan]).dtype

dtype('O')

### Nullable类型的性质

为了克服上述缺陷，pandas设计了一种新的缺失类型`pd.NA`以及三种`Nullable`**序列类型**：`Int`, `boolean`, `string`。在`Nullable`类型中存储缺失值，都会转为pandas内置的`pd.NA`类型。

In [31]:
pd.Series([np.nan, 1], dtype = 'Int64')

0    <NA>
1       1
dtype: Int64

In [32]:
# 对比
pd.Series([np.nan, 1])

0    NaN
1    1.0
dtype: float64

在实际处理数据时，可以在读入数据后先通过`covert_dypes`将序列转为`Nullable`类型。

In [33]:
# 原本类型
df.dtypes

Grade        object
Name         object
Gender       object
Height      float64
Weight      float64
Transfer     object
dtype: object

In [34]:
# 转换为Nullable序列
df.convert_dtypes().dtypes

Grade        string
Name         string
Gender       string
Height      float64
Weight        Int64
Transfer     string
dtype: object

### 缺失数据的计算和分组

进行加法（`sum`）和乘法（`prod`）时，缺失数据被分别视作0和1，实际上不参与计算：

In [35]:
s = pd.Series([2,3,np.nan,4,5])

In [36]:
s.sum()

14.0

In [37]:
s.prod()

120.0

使用累计函数时，自动**跳过**缺失值所处位置：

In [38]:
s.cumsum()

0     2.0
1     5.0
2     NaN
3     9.0
4    14.0
dtype: float64

在`groupby`和`get_dummies`中可以设置参数来增加**缺失类别**：

In [39]:
df_nan = pd.DataFrame({'category':['a','a','b',np.nan,np.nan],'value':[1,3,5,7,9]})
df_nan

,category,value
0,a,1
1,a,3
2,b,5
3,NaN,7
4,NaN,9


In [40]:
df_nan.groupby('category',dropna=False)['value'].mean()

category
a      2
b      5
NaN    8
Name: value, dtype: int64

In [41]:
pd.get_dummies(df_nan.category,dummy_na=True)

,a,b,NaN
0,1,0,0
1,1,0,0
2,0,1,0
3,0,0,1
4,0,0,1


## 练习

### Ex1: 缺失值与类别的相关性检验

In [42]:
from scipy.stats import chi2

In [43]:
df = pd.read_csv('data/missing_chi.csv')
df.head()

,X_1,X_2,y
0,NaN,NaN,0
1,NaN,NaN,0
2,NaN,NaN,0
3,43.0,NaN,0
4,NaN,NaN,0


In [44]:
df.isna().mean()

X_1    0.855
X_2    0.894
y      0.000
dtype: float64

In [45]:
df.y.value_counts(normalize=True)

0    0.918
1    0.082
Name: y, dtype: float64

In [46]:
E_1 = pd.DataFrame(np.array([df.loc[(df.X_1.isna()==m)&(df.y==n)].shape[0] for m in [False,True] for n in [0,1]]).reshape(2,2),index=['F','T'])

In [47]:
E_2 = pd.DataFrame(np.array([df.loc[(df.X_2.isna()==m)&(df.y==n)].shape[0] for m in [False,True] for n in [0,1]]).reshape(2,2),index=['F','T'])

In [48]:
F_1 = pd.DataFrame(np.array([(E_1.sum(1)[m] * E_1.sum(0)[n])/E_1.values.sum() for m in [0,1] for n in [0,1]]).reshape(2,2),index=['F','T'])

In [49]:
F_2 = pd.DataFrame(np.array([(E_2.sum(1)[m] * E_2.sum(0)[n])/E_2.values.sum() for m in [0,1] for n in [0,1]]).reshape(2,2),index=['F','T'])

In [50]:
S_1 = ((E_1 - F_1)**2 / F_1).values.sum()

In [51]:
S_2 = ((E_2 - F_2)**2 / F_2).values.sum()

利用scipy计算p值的方法如下：

In [52]:
chi2.sf(S_1,1)

0.9712760884395901

可见，`X_1`的缺失情况与标签正负存在相关关系。

In [53]:
chi2.sf(S_2,1)

7.459641265637543e-166

可见，`X_2`缺失情况与标签正负无关。

### Ex2: 用回归模型解决分类问题

我还没学过机器学习的相关知识，所以只能尝试理解答案的过程。

第1问：

In [54]:
from sklearn.neighbors import KNeighborsRegressor

In [55]:
df = pd.read_excel('data/color.xlsx')

In [56]:
df_dummies = pd.get_dummies(df.Color)

In [57]:
stack_list = []

In [58]:
# 分别对三种颜色的可能性进行预测
for col in df_dummies.columns:
    clf = KNeighborsRegressor(n_neighbors=6)
    clf.fit(df.iloc[:,:2], df_dummies[col])
    # 预测
    res = clf.predict([[0.8, -0.2]]).reshape(-1,1)
    stack_list.append(res)

In [59]:
stack_list

[array([[0.16666667]]), array([[0.33333333]]), array([[0.5]])]

In [60]:
# 找到概率最大的那一个对应的下标
code_res = pd.Series(np.hstack(stack_list).argmax(1))

In [61]:
# 完成预测
df_dummies.columns[code_res[0]]

'Yellow'

第2问：

In [62]:
from sklearn.neighbors import KNeighborsRegressor

In [63]:
df = pd.read_csv('data/audit.csv')

In [64]:
df.head(3)

,ID,Age,Employment,Marital,Income,Gender,Hours
0,1004641,38,Private,Unmarried,81838.00,Female,72
1,1010229,35,Private,Absent,72099.00,Male,30
2,1024587,32,Private,Divorced,154676.74,Male,40


In [65]:
# 生成备份
res_df = df.copy()

在进行缺失值插补前，先对原表进行处理：将`Marital`和`Gender`转为虚拟变量，将`Age`、`Income`和`Hours`标准化。

In [66]:
df = pd.concat([pd.get_dummies(df[['Marital', 'Gender']]),
                df[['Age','Income','Hours']].apply(lambda x:(x-x.min())/(x.max()-x.min())),
                df.Employment],1)     

In [67]:
df.head(3)

,Marital_Absent,Marital_Divorced,Marital_Married,Marital_Married-spouse-absent,Marital_Unmarried,Marital_Widowed,Gender_Female,Gender_Male,Age,Income,Hours,Employment
0,0,0,0,0,1,0,1,0,0.287671,0.168997,0.724490,Private
1,1,0,0,0,0,0,0,1,0.246575,0.148735,0.295918,Private
2,0,1,0,0,0,0,0,1,0.205479,0.320539,0.397959,Private


In [68]:
# 生成训练集，即那些Employment未缺失的样本 
X_train = df.loc[df.Employment.notna()]

In [69]:
# 生成测试集，即待插补的样本
X_test = df.loc[df.Employment.isna()]

之后的步骤和第一问类似：

In [70]:
df_dummies = pd.get_dummies(X_train.Employment)

In [71]:
stack_list = []

In [72]:
for col in df_dummies.columns:
    clf = KNeighborsRegressor(n_neighbors=6)
    clf.fit(X_train.iloc[:,:-1], df_dummies[col])
    res = clf.predict(X_test.iloc[:,:-1]).reshape(-1,1)
    stack_list.append(res)

In [73]:
code_res = pd.Series(np.hstack(stack_list).argmax(1))

In [74]:
code_res

0     2
1     0
2     4
3     4
4     4
     ..
95    4
96    4
97    4
98    4
99    4
Length: 100, dtype: int64

In [75]:
# 将位置编码转换为对应的职业
cat_res = code_res.replace(dict(zip(list(range(df_dummies.shape[0])),df_dummies.columns)))

In [76]:
# 将备份表格的职业缺失值用预测值替代
res_df.loc[res_df.Employment.isna(), 'Employment'] = cat_res.values

In [77]:
res_df.isna().sum()

ID            0
Age           0
Employment    0
Marital       0
Income        0
Gender        0
Hours         0
dtype: int64